In [ ]:
%%capture
# !git clone https://github.com/smfm-project/biota.git
!pip install scikit-image
# !pip uninstall biota -y
!pip install biota/

In [5]:
import os
import getpass
from pathlib import Path
from functools import partial

import biota
from biota import download as dw

from ipyleaflet import Marker
from ipywidgets import Output, Layout, HTML
from traitlets import Int, Bool, Float, link, Unicode, observe, List
import ipyvuetify as v
from sepal_ui import mapping as m
from sepal_ui import sepalwidgets as sw;

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

ResizeTrigger()

In [6]:
from ipywidgets import HTML

In [7]:
STYLES = """
<style>
.output_text pre, .widget-html{
    color : auto !important;
}

</style>
"""

_ = display(HTML(STYLES));

HTML(value='\n<style>\n.output_text pre, .widget-html{\n    color : auto !important;\n}\n\n</style>\n')

In [8]:
HEADER_TITLE = "SEPAL BIOTA (BETA)"
HEADER_TEXT = """
The BIOmass Tool for Alos (BIOTA) was developed by LTS International and the University of Edinburgh to calculate avove-ground biomass from L-band satallite data in dry forests and savanhas, as well as biomass change and forest degradation.
"""
OUTPUT_TITLE = "OUTPUT PROCESS"

In [12]:
class UI(v.Layout):
    
    USER = getpass.getuser()
    PARAMETER_FILE = os.path.join('/home', USER, 'SMFM_biota/biota/cfg/McNicol2018.csv')

    # Initial widgets
    lat = Float(0).tag(sync=True)
    lon = Float(-75).tag(sync=True)
    year_1 = Unicode().tag(sync=True)
    year_2 = Unicode().tag(sync=True)
    large_tile = Bool(True).tag(sync=True)
    grid = Int(5).tag(sync=True)
    
    # Process widgets
    forest_p = Bool(False).tag(sync=True)
    forest_ch = Bool(False).tag(sync=True)
    gamma0 = Bool(True).tag(sync=True)
    biomass = Bool(True).tag(sync=True)
    forest_cv = Bool(False).tag(sync=True)
    
    # Outputs
    true_cb = List([]).tag(sync=True)
#     display_vmodel = Unicode().tag(sync=True)
    
    def __init__(self, map_=None, **kwargs):
        
        # Widget classes
        self.class_ = "flex-column pa-2"
        self.row = True
        self.xs12 = True
        
        super().__init__(**kwargs)
        
        # Set up variables
        
        self.map_ = map_
        
        self.tile_1 = None
        self.tile_2 = None
        
        self.gamma0_tile = None
        self.agb_tile = None
        
        
        self._get_true_cb()
        
        
        # Set-up workspace
        self._workspace()

        # Output widgets
        self.ou_progress = Output()
        self.ou_display = Output()
        self.w_alert = sw.Alert()
        
        # 1. .Input widgets (Parameters)
        self.w_lat = v.TextField(disabled=True, label='Latitude',v_model=self.lat,)
        self.w_lon = v.TextField(disabled=True, label='Longitude', v_model=self.lon,)
        self.w_year_1 = v.TextField(label='Year 1', v_model=self.year_1,)
        self.w_year_2 = v.TextField(label='Year 2', v_model=self.year_2,)
        self.w_lg_tile = v.Checkbox(label='Large Tile', v_model=self.large_tile)
        self.w_grid = v.RadioGroup(v_model=self.grid,children=[
            v.Radio(label="1x1 grid", value=1),
            v.Radio(label="5x5 grid", value=5)
        ])
        
        # 2. Process input widgets (products)
        self.w_forest_p = v.Checkbox(label='Forest property', class_='pl-5', v_model=self.forest_p)
        self.w_forest_ch = v.Checkbox(label='Forest change', class_='pl-5', v_model=self.forest_ch)
        self.w_gamma0 = v.Checkbox(label='Gamma0', class_='pl-5', v_model=self.gamma0)
        self.w_biomass = v.Checkbox(label='Biomass', class_='pl-5', v_model=self.biomass)
        self.w_forest_cv = v.Checkbox(label='Forest cover', class_='pl-5', v_model=self.forest_cv)
        self.w_download = sw.Btn('Download images', class_='pl-5')
        
        # 3. Display inputs (maps)
        self.w_select_output = v.Select(items=self.true_cb, v_model=None)
        
        
        # Button widgets
        
        self.w_download = sw.Btn('Download images', class_='pl-5')
        self.btn_process = sw.Btn('Get outputs', class_='pl-5')
        self.btn_add_map = sw.Btn('Display', class_='pl-5')
        
        # Linked widgets
        link((self.w_lon, 'v_model'), (self, 'lon'))
        link((self.w_lat, 'v_model'), (self, 'lat'))
        link((self.w_year_1, 'v_model'), (self, 'year_1'))
        link((self.w_year_2, 'v_model'), (self, 'year_2'))
        link((self.w_lg_tile, 'v_model'), (self, 'large_tile'))
        link((self.w_grid, 'v_model'), (self, 'grid'))

        link((self.w_forest_p, 'v_model'), (self, 'forest_p'))
        link((self.w_forest_ch, 'v_model'), (self, 'forest_ch'))
        link((self.w_gamma0, 'v_model'), (self, 'gamma0'))
        link((self.w_biomass, 'v_model'), (self, 'biomass'))
        link((self.w_forest_cv, 'v_model'), (self, 'forest_cv'))
        
        link((self.w_select_output, 'items'), (self, 'true_cb'))
#         link((self.w_select_output, 'v_model'), (self, 'display_vmodel'))

        # Events
        self.w_download.on_event('click', self._download)
        self.btn_process.on_event('click', self._process)
        self.btn_add_map.on_event('click', self._display)
        
        
        self.children = [
            
            v.Card(children=[
                v.CardTitle(children=[HEADER_TITLE]), 
                v.CardText(children=[HEADER_TEXT])
            ]),
            
            v.Row(class_="d-flex flex-row ", xs12=True, md6=True,
               children=[
                    v.Col(
                        children=[
                            v.Card(class_='pa-4',children=[
                                self.w_lon, 
                                self.w_lat,
                                self.w_year_1,
                                self.w_year_2,
                                self.w_lg_tile,
                                self.w_grid,
                                self.w_download
                            ])

                        ]
                    ),
                    v.Col(
                        children=[
                            v.Card(class_='pa-2 justify-center', children=[
                                map_
                            ])
                        ]
                    ),
            ]),

            # Second row
            # Progress bar
            v.Card(class_="flex-row pa-2 mb-3", children=[
                self.w_alert,
                self.ou_progress,

            ]),
            
            # Third row
            # Process Checkboxes
#             
            v.Card(children=[
                v.CardTitle(children=[OUTPUT_TITLE]),
                v.CardText(class_="d-flex flex-row", children=[
                    v.Col(
                        children=[
                                self.w_forest_p,
                                self.w_forest_ch,
                        ]
                    ),
                    v.Col(
                        children=[
                                self.w_gamma0,
                                self.w_biomass,
                        ]
                    ),
                    v.Col(
                        children=[
                                self.w_forest_cv,
                                self.btn_process,
                        ]
                    ),
                ])
            ]),
            
            # Fourth row
            # Display Outputs
            
            v.Row(class_="d-flex flex-row ", xs12=True, md6=True,
               children=[
                    v.Col(
                        children=[
                            v.Card(class_='pa-4',children=[
                                self.w_select_output, 
                                self.btn_add_map,
                            ])

                        ]
                    ),
                    v.Col(
                        children=[
                            v.Card(class_='pa-2 justify-center', children=[
                                self.ou_display
                            ])
                        ]
                    ),
            ]), 
            
        ]


    # Add all True checkBoxes to a List
    # Inspect its change
    
    @observe('forest_p')
    def _observe_forest_p(self, change):
        self._get_true_cb()

    @observe('forest_ch')
    def _observe_forest_ch(self, change):
        self._get_true_cb()
        
    @observe('gamma0')
    def _observe_gamma0(self, change):
        self._get_true_cb()
        
    @observe('biomass')
    def _observe_biomass(self, change):
        self._get_true_cb()
        
    @observe('forest_cv')
    def _observe_forest_cv(self, change):
        self._get_true_cb()

        
    def _get_true_cb(self):

        labels = {
            'Forest property': self.forest_p,
            'Forest change': self.forest_ch,
            'Gamma0': self.gamma0,
            'Biomass': self.biomass,
            'Forest cover': self.forest_cv
        }

        self.true_cb = [k for k, v in labels.items() if v is True]

    def _process(self, widget, event, data):
                
        self.btn_process.toggle_loading()
        
        # Create tiles if years are selected.
        if self.year_1 : self.tile_1 = self._load_tile(int(self.year_1))
        if self.year_2 : self.tile_2 = self._load_tile(int(self.year_2))
            
        for process in self.true_cb:
            self.w_alert.reset()
            if process == 'Gamma0':
                self.w_alert.type_='info'
                self.w_alert.append_msg(f'Computing Gamma0')
                self.gamma0_tile = self.tile_1.getGamma0(polarisation = 'HV', units = 'decibels')
                self.w_alert.append_msg(f'Gamma0 computed and ready to display')
                self.w_alert.type_='success'
            elif process == 'Biomass':
                self.w_alert.append_msg(f'Computing Biomass')
                self.w_alert.type_='info'
                self.agb_tile = self.tile_1.getAGB()
                self.w_alert.append_msg(f'Biomass computed and ready to display')
        
        self.w_alert.type='success'
        self.btn_process.toggle_loading()
        
    def _display(self, widget, event, data):

        tiles = {
            'Gamma0': self.gamma0_tile,
            'Biomass': self.agb_tile,
        }
        tile_name = self.w_select_output.v_model

        tile = tiles[tile_name]        
        
        self.btn_add_map.toggle_loading()
        with self.ou_display:
            self.ou_display.clear_output()
            
            if tile_name == 'Biomass':
                self.tile_1._LoadTile__showArray(tile, title = 'AGB', cbartitle = 'tC/ha', vmin = 0, vmax = 40, cmap = 'YlGn')
            
            elif tile_name == 'Gamma0':
                polarisation = 'HV'
                units = 'decibels'
                vmin, vmax = -20, -10
                self.tile_1._LoadTile__showArray(tile, 
                                                 title = 'Gamma0 %s'%polarisation, 
                                                 cbartitle = units, 
                                                 vmin = vmin, 
                                                 vmax = vmax, 
                                                 cmap = 'Greys_r')
            
        self.btn_add_map.toggle_loading()
        
        
        
    
    def _round(self, x):
        
        return self.grid * round(x/self.grid)
    
    def _workspace(self):
        """ Creates the workspace necessary to store the data

        return:
            Returns environment Paths

        """

        base_dir = Path(os.path.join('/home', self.USER))

        root_dir = base_dir/'smfm'
        data_dir = root_dir/'data'
        output_dir = root_dir/'outputs'

        root_dir.mkdir(parents=True, exist_ok=True)
        data_dir.mkdir(parents=True, exist_ok=True)
        output_dir.mkdir(parents=True, exist_ok=True)

        self.root_dir = root_dir
        self.data_dir  = data_dir
        self.output_dir = output_dir
        

    
    def _load_tile(self, year):

        try:
            tile = biota.LoadTile(str(ui.data_dir), 
                                       ui._round(ui.lat), 
                                       ui._round(ui.lon), 
                                       year,
                                       parameter_file = ui.PARAMETER_FILE,
                                       lee_filter = True, 
                                       forest_threshold = 15., 
                                       area_threshold = 1, 
                                       output_dir = str(ui.output_dir))
            return tile
        except Exception as e:

            self.w_alert.add_msg(f'{e}', type_='error')
            
    def _download(self, widget, event, data):
        
        self.w_download.toggle_loading()
        with self.ou_progress:
            
            try:
                self.ou_progress.clear_output()
                self.w_alert.reset()

                years = [int(year) for year in [self.year_1, self.year_2] if year]
                lat = self._round(self.lat)
                lon = self._round(self.lon)

                if not years:
                    self.w_alert.add_msg('Please select at least one year.', type_='error')
            
                self.w_alert.add_msg(f'Downloading...', type_='info')
            
                for y in years:
                    dw.download(lat, 
                                lon, 
                                y, 
                                large_tile=self.large_tile, 
                                output_dir=self.data_dir, 
                                verbose=True)

                self._decompress()
            
            except Exception as e:
                self.w_alert.add_msg(f'{e}', type_='error')
                
        self.w_download.toggle_loading()

    def _decompress(self):
        
        tar_files = list(self.data_dir.glob('*.tar.gz'))

        for tar in tar_files:
            if not os.path.exists(os.path.join(self.data_dir, tar.name[:-7])):
                self.w_alert.add_msg(f'Decompressing {tar.name}...', type_='info')
                dw.decompress(str(tar))
                self.w_alert.add_msg(f'All the images were succesfully unzipped.', type_='success')

In [13]:
def _return_coordinates(self, **kwargs):
    if kwargs.get('type') == 'click':

        # Remove markdown if there is one
        map_.remove_last_layer()

        lat, lon = kwargs.get('coordinates')

        map_.add_layer(Marker(location=kwargs.get('coordinates')))
        ui.lat = round(lat,2)
        ui.lon = round(lon,2)
# Map
map_ = m.SepalMap()
ui = UI(map_=map_)
map_.on_interaction(partial(_return_coordinates, ui))

In [14]:
ui

UI(children=[Card(children=[CardTitle(children=['SEPAL BIOTA (BETA)']), CardText(children=['\nThe BIOmass Tool…